Connected to base (Python 3.10.10)

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from skimage import transform
df_coords = pd.read_csv("intermediate/coordinations_new_beads.csv")
df_coords = df_coords.astype({"by":"str","detector":"str","color":"str"})
# fill the camera coordinates that didn't show beads

list_df = [df_coords]
for fov,by in ((2,"detector"),(2,"color"),(3,"detector")):
    stack = []
    for color in ["DAPI","FITC","YFP","TRITC"]:
        df_camera = df_coords.loc[
            (
                df_coords["FOV"].eq(fov)
              & df_coords["by"].eq(by)
              & df_coords["color"].eq(color)
            ),
            ['x','y']
        ].dropna()
        if len(df_camera) == 0:
            continue 
        stack.append(df_camera.to_numpy())
    ave_camera = np.mean(np.stack(stack,axis=0),axis=0)
    list_df.append(pd.DataFrame({
        "FOV": fov,
        "by": by,
        "detector": "camera",
        "color": "CFP",
        "index": np.arange(ave_camera.shape[0]),
        "y": ave_camera[:,1],
        "x": ave_camera[:,0]
    }))
    if fov==2 and by=="color":
        list_df.append(pd.DataFrame({
            "FOV": fov,
            "by": by,
            "detector": "camera",
            "color": "YFP",
            "index": np.arange(ave_camera.shape[0]),
            "y": ave_camera[:,1],
            "x": ave_camera[:,0]
        }))

df_filled = pd.concat(list_df,ignore_index=True)

In [2]:
df_filled

,FOV,by,detector,color,index,label,y,x
0,2,detector,spectral,blue,0,7.0,34.767673,70.573483
1,2,detector,spectral,blue,1,15.0,81.319578,42.509932
2,2,detector,spectral,blue,2,16.0,88.044346,49.490895
3,2,detector,spectral,blue,3,10.0,63.702043,87.960067
4,2,detector,spectral,blue,4,14.0,74.377717,86.692644
...,...,...,...,...,...,...,...,...
1183,3,detector,camera,CFP,43,NaN,705.112380,520.277644
1184,3,detector,camera,CFP,44,NaN,730.080643,479.478253
1185,3,detector,camera,CFP,45,NaN,725.627495,613.525751
1186,3,detector,camera,CFP,46,NaN,747.422250,283.058395
